In [1]:
%pip install -U transformers
%pip install accelerate
%pip install huggingface_hub
%pip install pillow


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Load images
image1 = Image.open("/home/Screenshot 2024-11-15 124654.png")

# Initialize processor and model
# processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Base")
# model = AutoModelForVision2Seq.from_pretrained(
#     "HuggingFaceTB/SmolVLM-Base",
#     torch_dtype=torch.bfloat16,
#     # _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
# ).to(DEVICE)

processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
model = AutoModelForVision2Seq.from_pretrained("HuggingFaceTB/SmolVLM-Instruct").to(DEVICE)





Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


In [16]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": """Extract all visible text from the entire image, including headers, footers, and any text near the edges or corners. 
                                    Pay special attention to identifying handwritten text, even if it is small, faint, or located in less prominent areas. 
                                    Highlight handwritten text using **bold** format. Present all extracted text as markdown format, including formatting, line breaks, and special characters."""}
        ]
    },
]
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to(DEVICE)
# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=2000)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)
print(generated_texts[0])

User:<image>Extract all visible text from the entire image, including headers, footers, and any text near the edges or corners. 
                                    Pay special attention to identifying handwritten text, even if it is small, faint, or located in less prominent areas. 
                                    Highlight handwritten text using **bold** format. Present all extracted text as markdown format, including formatting, line breaks, and special characters.
Assistant: **Date & Time: 18-10-22**
**Evaluation/Progress Notes:**
- Blood sugar level: 140
- Forks: 1350
- Bmi: 9.1
- Pain: 10
- Psychological/sleep: 11
- Elimination: 1
- Personal hygiene: 6
- Signature: C30

**Ward:**

**Evaluation/Progress Notes:**
- Blood sugar level: 140
- Forks: 1350
- Bmi: 9.1
- Pain: 10
- Psychological/sleep: 11
- Elimination: 1
- Personal hygiene: 6
- Signature: C30

**Ward:**

**Evaluation/Progress Notes:**
- Blood sugar level: 140
- Forks: 1350
- Bmi: 9.1
- Pain: 10
- Psychological/sleep: